In [1]:
import numpy as np
import pandas as pd

# df = pd.read_csv("FrankensteinFullForm.csv")
# df = df.as_matrix()
# print(df.shape)
# labels = pd.read_csv("LabelFrankensteinFullForm.csv", header = None)
# labels = labels.as_matrix()

<h1>Load Dataset</h1>

In [2]:
import pickle
from numpy import genfromtxt
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
import sklearn

TRAIN_RATIO = 0.8
AMOUNT_OF_CLASSES = 7


def loadDataset(path):
    inputdataset = (genfromtxt(path, delimiter=','))[1:,:]
    dataset = normalize(inputdataset)
    trainingAmount = int(TRAIN_RATIO * dataset.shape[0])
    training_dataset = dataset[0:trainingAmount,:]
    test_dataset = dataset[trainingAmount:,:]
    print("data: ", dataset.shape)
    return training_dataset.astype(float), test_dataset.astype(float)

def loadLabel(data_path, labels_path):
    dataset = (genfromtxt(data_path, delimiter=','))[1:,:]
#     dataset = normalize(dataset)
    trainingAmount = int(TRAIN_RATIO * dataset.shape[0])
    #     labels = pickle.load(INIT_PATH + path)
    with open(labels_path, 'rb') as handle:
        labels = pickle.load(handle)
    ###### TEMPORARY
    del labels[-1]
    ######
    all_labels = np.zeros((len(labels), AMOUNT_OF_CLASSES))
#     training_labels = np.zeros((trainingAmount, AMOUNT_OF_CLASSES))
#     test_labels = np.zeros((len(labels) - trainingAmount, AMOUNT_OF_CLASSES))
    label_counter = 0
    per_label = [0] * AMOUNT_OF_CLASSES
#     print(labels)
    for label in labels:
        tmp = np.zeros((1,AMOUNT_OF_CLASSES))
        if (label == "static"):
            tmp[0,0] = 1
            per_label[0] += 1
        elif (label == "move"):
            tmp[0,1] = 1
            per_label[1] += 1
        elif (label == "rot"):
            tmp[0,2] = 1
            per_label[2] += 1            
        elif (label == "frot"):
            tmp[0,3] = 1
            per_label[3] += 1
        elif (label == "light"):
            tmp[0,4] = 1
            per_label[4] += 1
        elif (label == "dynamiclight"):
            tmp[0,5] = 1
            per_label[5] += 1
        else:
            tmp[0,6] = 1
            per_label[6] += 1
        all_labels[label_counter,:] = tmp
#         if(label_counter < trainingAmount):
#             training_labels[label_counter,:] = tmp
#         else:
#             test_labels[label_counter - trainingAmount,:] = tmp
        label_counter += 1
    dataset, all_labels = shuffle(dataset, all_labels, random_state=0)
    return assignData(dataset, all_labels)
    
#     train_dataset = dataset[0:trainingAmount,:]
#     test_dataset = dataset[trainingAmount:,:]
#     print("data: ", dataset.shape)
    
#     train_labels = all_labels[:trainingAmount,:]
#     test_labels = all_labels[trainingAmount:,:]
    
#     print(", ".join([definition[i] + ": " + str(per_label[i]) for i in range (len(per_label))]))
#     return train_dataset.astype(float), test_dataset.astype(float),train_labels, test_labels

In [3]:
def assignData(data,labels):
    UNKNOWN_AMOUNT = 5
    train_amount = int(data.shape[0] * TRAIN_RATIO)
    AMOUNT_PER_CLASS = int((data.shape[0] - UNKNOWN_AMOUNT) * TRAIN_RATIO)
    class_count = [0] * train_amount
    train_data = []
    test_data = []
    train_labels = []
    test_labels = []

    for i in range (data.shape[0]):
        if(len(train_data) == train_amount):
            test_data.append(data[i])
            test_labels.append(labels[i])
        else:
            index = np.argmax(data[i])
            if(index == AMOUNT_OF_CLASSES - 1):
                test_data.append(data[i])
                test_labels.append(labels[i])
            if( class_count[index] < AMOUNT_PER_CLASS) :
                train_data.append(data[i])
                train_labels.append(labels[i])
                class_count[index] += 1
            else:
                test_data.append(data[i])
                test_labels.append(abels[i])
    return np.asarray(train_data), np.asarray(test_data), np.asarray(train_labels), np.asarray(test_labels)
                    

In [4]:
train_data, test_data, train_labels, test_labels = loadLabel("../unnormFrankensteinFullForm.csv","LabelFrankensteinFullForm.p")
print(train_labels.dtype)
print(train_data.shape)
print(test_data.shape)
print(train_labels.shape)
print(test_labels.shape)
# print(train_labels)
print(np.sum(train_labels,axis=0))

float64
(110, 10)
(28, 10)
(110, 7)
(28, 7)
[19. 16. 17. 17. 18. 18.  5.]


In [5]:
with open("../unnormFrankensteinFullForm.csv") as f:
    lines = (line for line in f if not line.startswith('#'))
    dataset = np.loadtxt(lines, delimiter=',', skiprows=1)
definition = ['static', 'move', 'rot', 'frot', 'light', 'dynamiclight','unknown']
scaler = StandardScaler(copy=True)
scaler.fit(dataset)

def normalize(dataset):
    datasetTmp = scaler.transform(dataset)
    return datasetTmp
dataset = normalize(dataset)

In [6]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from keras import optimizers

/home/bon/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
model = Sequential()

model.add(Dense(7, activation='linear', input_dim=10))
model.add(Dense(128,activation='linear'))
model.add(Dense(256, activation='linear'))
model.add(Dense(128, activation='linear'))
model.add(Dense(64, activation='linear'))
model.add(Dense(7, activation='softmax'))

In [8]:
# opti = optimizers.SGD(lr=0.0002, decay=1e-6, momentum=0.9, nesterov=True)
opti =optimizers.Adam(beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='binary_crossentropy', optimizer=opti, metrics=['accuracy'])
# model.compile(loss='MSE', optimizer=opti, metrics=['accuracy'])

In [ ]:
model.fit(normalize(train_data),train_labels,batch_size=1, epochs=500, verbose=1)

In [8]:
score = model.evaluate(normalize(test_data), test_labels, verbose=0)
print(score)

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.

In [9]:
# serialize model to YAML
model_yaml = model.to_yaml()
with open("model23.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("model23.h5")
print("Saved model to disk")


Saved model to disk


In [10]:
from keras.models import model_from_yaml
# load YAML and create model
yaml_file = open('model20.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("model20.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
# loaded_model.compile(loss='MSE', optimizer=opti, metrics=['accuracy'])
loaded_model.compile(loss='binary_crossentropy', optimizer=opti, metrics=['accuracy'])
# score = loaded_model.evaluate(test_data, test_labels, verbose=0)
score = loaded_model.evaluate(normalize(test_data), test_labels, verbose=0)

print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
acc: 90.31%


In [11]:
definition = ['static', 'move', 'rot', 'frot', 'light', 'dynamiclight','unknown']

for test_num in range (20):
    lol = normalize(train_data[test_num].reshape(1,-1))
    lol_label = train_labels[test_num]
    # print(lol.shape)
    # print(lol, lol_label)
    prediction = loaded_model.predict(lol.reshape(1,10))
    print(definition[np.argmax(prediction[0])])
    print(definition[np.argmax(lol_label)])
    print("-----------------------------------------------")

static
static
-----------------------------------------------
frot
frot
-----------------------------------------------
static
static
-----------------------------------------------
rot
rot
-----------------------------------------------
frot
rot
-----------------------------------------------
static
dynamiclight
-----------------------------------------------
dynamiclight
move
-----------------------------------------------
move
move
-----------------------------------------------
rot
rot
-----------------------------------------------
frot
frot
-----------------------------------------------
rot
frot
-----------------------------------------------
rot
rot
-----------------------------------------------
dynamiclight
dynamiclight
-----------------------------------------------
light
light
-----------------------------------------------
frot
frot
-----------------------------------------------
dynamiclight
dynamiclight
-----------------------------------------------
move
move
----------

In [12]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np

BUFFER_SIZE = 100
WIDTH = 1280
HEIGHT = 720
video_count = 0

In [13]:
def calculate_n_of_hsv(frame_sq,n):
    sum_n = 0
    var_n = 0
    for frame in frame_sq:
        var_n += np.var(frame[:,:,n])# / (np.mean(frame[:,:,n]) * 1
        sum_n += np.sum(frame[:,:,n])

    avg_n = sum_n/ (len(frame_sq) * frame_sq[0].shape[0] * frame_sq[0].shape[1])
    return np.array([avg_n ,var_n])
def calculate_comp(frame_sq):
    sum_comp = 0
    for i in range(len(frame_sq)):
        if(i != 0):
            sum_comp += compare_img(frame_sq[i], frame_sq[i-1])
#         if(i == 1): print(compare_img(frame_sq[i], frame_sq[i-1]))
    sum_comp /= (len(frame_sq) - 1)
    return sum_comp
def calculate_opticFlow(frame_sq):
    prvs = cv2.resize(frame_sq[0],(0,0),fx=0.5,fy=0.5)
    mag_sum = 0
    ang_sum = 0
    var_mag = 0
    var_ang = 0
    for i in range(1,len(frame_sq)):
        frame2 = frame_sq[i]
        next = cv2.resize(frame2, (0,0), fx=0.5, fy=0.5) 
        flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
        mag = cv2.normalize(mag,None,0,1,cv2.NORM_MINMAX)
        ang = np.absolute(cv2.normalize(ang,None,-1,1,cv2.NORM_MINMAX))
        mag_sum = np.sum(mag)
        ang_sum = np.sum(ang)
        var_mag += np.var(mag)
        var_ang += np.var(ang)
    mag_sum /= (BUFFER_SIZE * WIDTH * HEIGHT)
    ang_sum /= (BUFFER_SIZE * WIDTH * HEIGHT)
    var_mag /= BUFFER_SIZE
    var_ang /= BUFFER_SIZE
    return np.array([mag_sum, ang_sum, var_mag, var_ang])

In [14]:
def compare_img(img1,img2):
    img1 = normalize_gray(img1)
    img2 = normalize_gray(img2)
    diff = img1 - img2
    m_norm = np.sum(abs(diff))
#     return m_norm
    return 0
def normalize_frame(frame):
    return frame/255

def normalize_gray(frame):
    rng = np.max(frame) - np.min(frame)
    amin = np.min(frame)
    return (frame-amin)/rng #range from [0,1]
def createDataset(data):
    dataset = data[0].copy()
    data.remove(data[0])
    for datum in data:
        dataset = np.append(dataset, datum, axis = 1)
    return dataset
def createTSV(dataset,filename):
    dataset = (dataset.copy()).astype(str)
#     datasetName = "dataset" + str(dataset_no) + ".tsv"
    datasetName = filename + ".tsv"
    file = open(datasetName, "w")
#     file.write("h\ts\tv\tframe_edges_comp\tframe_comp\n")
    for i in range(dataset.shape[0]):
        file.write("\t".join(dataset[i,:].tolist()))
        file.write("\n")
#         if(i + 1 != dataset.shape[0]):
#             file.write("\n")
    file.close()
    return
def createTSVLabel(dataset,filename):
    dataset = (dataset.copy()).astype(str)
#     datasetName = "label" + str(dataset_no) + ".tsv"
    datasetName = filename
    file = open(datasetName, "w")
    file.write("havg\thv\tsavg\tsv\tvavg\tvv\tmagavg\tmagv\tangavg\tangv\n")
    for i in range(dataset.shape[0]):
        file.write("\t".join(dataset[i,:].tolist()))
        file.write("\n")
#         if(i + 1 != dataset.shape[0]):
#             file.write("\n")
    file.close()
    return


In [63]:
def _normalize(frame):
#     print("input: ", frame)
    new_frame = scaler.transform(frame.reshape(1,-1))
#     new_frame = frame.reshape(1,-1)
#     print("output: ",new_frame)
    return new_frame

In [108]:
def process(frame_sq,frame_sq_gray):
#     print(len(frame_sq))
#     print(frame_sq[0].shape)
#     print(frame_sq[0][1,2])
    h = calculate_n_of_hsv(frame_sq,0)
#     print("h:",h)
    s = calculate_n_of_hsv(frame_sq,1)
#     print("s:",s)
    v = calculate_n_of_hsv(frame_sq,2)
#     print("v:",v)
    opticFlow = calculate_opticFlow(frame_sq_gray)
#     print("of:",opticFlow)
    return _normalize(np.append(h,np.append(s,np.append(v,opticFlow))))
def predict(output):
    prediction = loaded_model.predict(output)
    print("confidence: " + str(prediction))
    return(definition[np.argmax(prediction[0])])

In [109]:
import numpy as np
import cv2
import sys
from sklearn import preprocessing

frame_sq = []
frame_sq_gray = []
BUFFER_SIZE = 100
cap = cv2.VideoCapture("../doit3/49.avi")
video_count = 0
while(True):
    ret, frame = cap.read()
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)  
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    if(video_count == 0):
        WIDTH = int(cap.get(3))
        HEIGHT = int(cap.get(4))
        video_count += 1
    frame_sq.append(normalize_frame(hsv))
    frame_sq_gray.append(normalize_frame(gray))
    if(len(frame_sq) >= BUFFER_SIZE):
        output = process(frame_sq, frame_sq_gray)
        predictt = predict(output)
        sys.stdout.write("\r" + predictt)
        sys.stdout.flush()        
#         del frame_sq[0]
        frame_sq = []
        frame_sq_gray = []
        break
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

confidence: [[8.73614177e-02 7.48786628e-01 1.41017929e-01 2.28340551e-02
  7.34770256e-20 2.21918395e-08 1.32927935e-08]]
move